## Random Forest

###### author: *Thiago V. Lemes*

###### e-mail: *lemes_vilarinho@yahoo.com.br*

###### LinkedIn: *https://www.linkedin.com/in/thiago-vilarinho-lemes-b1232727/*

In [ ]:
# Instalando a biblioteca dash
!pip install dash

In [ ]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
# Máscara para os números decimais
pd.options.display.float_format='{:.2f}'.format

In [ ]:
# Importando os dados
df = pd.read_csv('/content/drive/MyDrive/Dataset/california_housing_train.csv')
df.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.00,5612.00,1283.00,1015.00,472.00,1.49,66900.00
1,-114.47,34.40,19.00,7650.00,1901.00,1129.00,463.00,1.82,80100.00


In [ ]:
colunas = [
  'longitude',
  'latitude',
  'idade_media_casas', # housing_median_age
  'total_quartos', # total_rooms
  'total_suites', # total_bedrooms
  'populacao', # population
  'familias', # households
  'renda_media', # median_income
  'valor_medio_casa' # median_house_value
]

In [ ]:
# Renomeando colunas
df.set_axis(colunas, axis=1, inplace=True)

## Machine Learning - Treinando o modelo

In [ ]:
# Bibliotecas do sklearn
from sklearn import ensemble
from sklearn.model_selection import train_test_split # Divisor do dataset para treino e validação
from sklearn.metrics import mean_squared_error # erro quadrático médio
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Filtrando todos os dados das features menos a 'valor_medio_casa' que é a target
# Features
x = df.drop('valor_medio_casa', axis=1)
x.head(2)

,longitude,latitude,idade_media_casas,total_quartos,total_suites,populacao,familias,renda_media
0,-114.31,34.19,15.00,5612.00,1283.00,1015.00,472.00,1.49
1,-114.47,34.40,19.00,7650.00,1901.00,1129.00,463.00,1.82


In [ ]:
# Filtrando todos os dados da target 'valor_medio_casa'
# Target
y = df.valor_medio_casa
y[1]

80100.0

In [ ]:
# Filtrando as amostras para treino e validação 
# train_test_split - serve para dividir o array
# x - Features
# y - Target
# test_size - define em % o tamanho do conjunto de teste

xtr, xval, ytr, yval = train_test_split(x, y, test_size=0.5, random_state=0)

In [ ]:
# Criando a arvore de decisão
# random_state - semente da função random
# n_jobs - número de processadores, neste caso -1 utiliza todos os processadores

# Otimização
# min_samples_leaf - quantos exemplos em cada nó final
# n_estimators - número de arvores, por padrao é 100
arvores = ensemble.RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=1000, min_samples_leaf=2)  

# Treinando o algoritmo
arvores.fit(xtr, ytr)

RandomForestRegressor(min_samples_leaf=2, n_estimators=1000, n_jobs=-1,
                      random_state=0)

In [ ]:
# Teste
p = arvores.predict(xval)

# Erro quadrático médio
np.sqrt(mean_squared_error(yval, p))

50486.16517182602

In [ ]:
# Testes realizados

# defaul -> 50822.200850256515
# n_estimators=200  -> 50633.64512666656
# n_estimators=300  -> 50616.12238341953
# n_estimators=1000 -> 50460.51621177114
# n_estimators=300, min_samples_leaf=2 -> 50653.35917991981
# n_estimators=1000, min_samples_leaf=2 -> 50486.16517182602

In [ ]:
# Entendendo Random Forest
# Amostra com reposição - bootstrap sampling = baggging - pode sortear o mesmo número

In [ ]:
correlacao = df[['idade_media_casas', 'total_quartos',	'total_suites',	'renda_media', 'valor_medio_casa']].corr()

In [ ]:
correlacao

,idade_media_casas,total_quartos,total_suites,renda_media,valor_medio_casa
idade_media_casas,1.00,-0.36,-0.32,-0.12,0.11
total_quartos,-0.36,1.00,0.93,0.20,0.13
total_suites,-0.32,0.93,1.00,-0.01,0.05
renda_media,-0.12,0.20,-0.01,1.00,0.69
valor_medio_casa,0.11,0.13,0.05,0.69,1.00


In [ ]:
# Função do Gráfico do Heatmap - Mapa de calor
def heatmap_graphic(dataframe, width = 1000, height = 500, title=' '):
  fig = px.imshow(
    dataframe, 
    text_auto=True, # No centro
    width = width,
    height = height,
    title = title
  )
  fig.update_layout(font=dict(size = 12)) # Tamanho da fonte
  fig.update_coloraxes(showscale=False) # Remove a barra escalar de cores
  fig.show()

In [ ]:
# Gerando o gráfico
heatmap_graphic(correlacao, title='Gráfico Heatmap - Mapa de Calor')

In [ ]:
# Função do Gráfico de Scatter
# Tipos de theme = "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"
def scatter_graphic(dataframe, x, y, x_label = x, y_label = y, color=None, color_discrete_sequence = None, title=' ', 
                    template=None, size=None, symbol=None, width = 1000, height = 400):
  fig = px.scatter(
      dataframe,
      x = x,
      y = y, 
      color = color,                  # Muda as cores dos markers
      color_discrete_sequence = color_discrete_sequence, # Diversar cores
      size = size,                    # Tamanho das bolhas
      title = title,                  # Nome do Título
      template = template,            # Muda o tema
      trendline="ols",                # Linha de Regressão Linear
      symbol = symbol,                # Simbolo
      width = width,                  # Largura
      height = height,                # Altura
      labels={
          'renda_media': 'Renda Média',
          'valor_medio_casa': 'Valor Médio Casa',
      }
  )
  fig.update_xaxes(title = x_label) 
  fig.update_yaxes(title = y_label)
  fig.update_coloraxes(showscale=False) # Remove a barra escalar de cores
  
  fig.show()

In [ ]:
# Gerando o gráfico
scatter_graphic(
    df, 
    x = 'renda_media', 
    y = 'valor_medio_casa',
    x_label='Renda Média', 
    y_label = 'Valor Médio Casa', 
    title='Gráfico de Scatter', 
    size='renda_media',
    template = 'seaborn'
);

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
